In [ ]:
pip install -r requirements_all.txt

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import random as rd
from tkinter import * 
import openpyxl
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import time
import nest_asyncio
nest_asyncio.apply()


In [ ]:
#path folder
path=Path('') 
folder_LCI='Data_LCI'
path_LCI=path / folder_LCI
folder_python='Data_python'
path_python= path / folder_python

file_step1='LCI_modelling.xlsx'
path_step1= path_LCI / file_step1
file_step2='LCI_modelling_step2.xlsx'
path_step2= path_LCI / file_step2
file_step3='LCI_modelling_step3_BW.xlsx'
path_step3= path_LCI / file_step3

file_ei='20240710_LCI_BackgroundModelling_Guidelines&Data_v1.xlsx'
path_ei= path_python / file_ei
sheet_ei='Activities List (cut-off)'

#file_LCI_step0='LCI_Review&Compilation_W_v1.xlsx' #TO CHANGE
path_step0= path_LCI / file_LCI_step0

file_mapping='unit_name_ecoinvent.xlsx'
path_mapping= path_python / file_mapping

In [3]:

i=0
list_name_columns_LCI=["Flow\n(As reported by the authors)",	"Type of flow",	"Classification of flows",	"Value",	
                       "Unit\n(As reported by the authors)",	 "Reference flow\n(to which the LCI is reported)",
                       "Associated unit operation\n(as reported by the authors)",	"Order of the unit operation(s) in the process chain", 
                       "Corresponding life cycle stage","Pedigree - Reliability", "Pedigree - Completeness", "Pedigree - Temporal correlation",
                       "Pedigree - Geographical correlation", "Pedigree - Further technological correlation"]
list_name_add_blue=["Ei - Activity name",  "Elementary flow",	"Ei - Reference product",	"Elementary flow category", "Elementary flow sub-category",
                    "Ei - Location", "Unit", "Modified value to ei unit", "modified value (ref flow BW activity)", "modified value allocated flow"  ,	"Comment on calculations"]

def create_blue(path0, sheet_names_list,path1,i=0):
    for sheet in sheet_names_list:
        data_LCI_PRi=pd.read_excel(path_step0,sheet_name=sheet,header=[0])
        columns_need=data_LCI_PRi.loc[:,list_name_columns_LCI].copy()
        columns_need=columns_need.reindex(columns=[*columns_need.columns.tolist(), *list_name_add_blue])
        i+=1
        with pd.ExcelWriter(path_step1,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:  
            columns_need.to_excel(writer, sheet_name=sheet,index=False) #'PR' + str(i) + '_LCI_Modelling',

In [10]:
def on_button_clicked(b):
    global clicked
    clicked = True
    # with output:
    #     print("button clicked")
    #     print(clicked)

SyntaxError: invalid syntax (59974100.py, line 3)

In [8]:
#import pandas as pd

dict_flow_efcat=pd.read_excel(path_mapping,sheet_name='EI_flow_cat',header=None)#,index_col=0,header=None)
dict_flow_efcat=dict(dict_flow_efcat.values)

dict_unit=pd.read_excel(path_mapping,sheet_name='unit_ei_BW')
dict_unit=dict(dict_unit.values)
ecoinvent_data=pd.read_excel(path_ei,sheet_name=sheet_ei,header=[0])

ei_ef_data=pd.read_excel(path_ei,sheet_name='Elementary Exchanges List',header=[0])




def fill_ei_product(num_PR,sheet_names_list_model,dict_flow_efcat,dict_unit,ei_ef_data):

    
    for sheet in sheet_names_list_model:
    
        data_LCI_PRi_model=pd.read_excel(path_step1,sheet_name=sheet,header=[0]) #'PR'+ str(num_PR) +'_LCI_Modelling'
    
        print( 'The production route is ' + str(sheet))
        
        
        data_LCI_PRi_model_Activity=data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==data_LCI_PRi_model['Ei - Activity name']]
        list_activity=data_LCI_PRi_model_Activity['Ei - Activity name'].drop_duplicates()
        
        pb=False
        
        act_fill_later=[]
        
        prod_later=[]
        unit_later=[]
        count=0
        for activity in list_activity:
            #print(fill)
            #find the reference product + unit
            product_name=ecoinvent_data.loc[ecoinvent_data['Activity Name']==activity].loc[:,'Reference Product Name'].drop_duplicates()
            unit=ecoinvent_data.loc[ecoinvent_data['Activity Name']==activity].loc[:,'Unit'].drop_duplicates()
            if len(product_name)==0:
                print( "FOR THIS ACTIVITY :" + str(activity)+ " THERE IS NO CORRESPONDING IN THE ECOINVENT DATABASE")
                #pb=True
            if len(product_name)>1:
                count+=1
                list_dict_product=list(dict(product_name).values())
        
                sel_prod=widgets.RadioButtons(
                    options=list_dict_product,
                    #    value='pineapple', # Defaults to 'pineapple'
                    #    layout={'width': 'max-content'}, # If the items' names are long
                        description="Choose the best option for the activity: " + str(activity),
                        disabled=False,
                        layout={'width': 'max-content'}
                    )
        
                display(sel_prod)
                act_fill_later.append(activity)
        
                # print("error more than 1 product for the activity")
                # break
                #prod_later.append(sel_prod.value)
            if len(unit)>1:
                list_dict_unit=list(dict(unit).values())
        
                sel_unit=widgets.RadioButtons(
                    options=list_dict_unit,
                    #    value='pineapple', # Defaults to 'pineapple'
                    #    layout={'width': 'max-content'}, # If the items' names are long
                        description="Choose the best option for the activity: " + str(activity),
                        disabled=False,
                        layout={'width': 'max-content'}
                    )
        
                display(sel_unit)
        
                # print("error more than 1 product for the activity")
                # break
        
                
                #unit_later.append(sel_unit.value)
        
                
                # Create an instance of tkinter frame or window
                #unit=pd.Series(data=radio.get())
            if len(product_name)>1:
                button = widgets.Button(description = "Click when good")
                output = widgets.Output()
                display(button, output)
                global clicked
                clicked = False
                def on_button_clicked(b):
                    global clicked
                    clicked = True
                    # with output:
                    #     print("button clicked")
                
                button.on_click(on_button_clicked)

                #print(clicked)
                with ui_events() as poll:
                    while not clicked:
                        #print(clicked)
                        poll(10) # poll queued UI events including button
                        time.sleep(1) # wait for 1 second before checking again
                
                unit_later.append(sel_unit.value)
                prod_later.append(sel_prod.value)
                #print('python waited for the button click')
                # _check_id_button=widgets.Button(description='Values selected')
        
                # display(_check_id_button)
                # def _check_id(b):
                #     loop = asyncio.get_event_loop()
                #     result = loop.run_until_complete(contract.evaluate_transaction('queryUnprotectedEntry', [_id_info.value], user))
                #     unit_later.append(sel_unit.value)
                #     prod_later.append(sel_prod.value)
                # _check_id_button.on_click(_check_id)
        
                                                     
                # btn=widgets.Button(description='Values selected')
                # display(btn)
                
                # def fill(b):
                #     unit_later.append(sel_unit.value)
                #     prod_later.append(sel_prod.value)
                #     print('aaaaaaaa')
                
                # btn.on_click(fill)
            if len(product_name)==1 :
                index=data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==activity].index
                #data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==activity].loc[:,'Ei - Reference product']=product_name
                data_LCI_PRi_model.loc[index,'Ei - Reference product']=str(product_name.values[0])
                data_LCI_PRi_model.loc[index,'Unit']=dict_unit[str(unit.values[0])]
                data=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']==activity) & (data_LCI_PRi_model['Ei - Reference product']==str(product_name.values[0]))].loc[:,'Ei - Location']
                list_country=ecoinvent_data.loc[(ecoinvent_data['Activity Name']==activity) & (ecoinvent_data['Reference Product Name']==str(product_name.values[0]))].loc[:,'Geography']
                # data_LCI_PRi_model.loc[index,'Ei - Reference product']=product_name
                # data_LCI_PRi_model.loc[index,'Unit']=unit
                # data=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']==activity) & (data_LCI_PRi_model['Ei - Reference product']==product_name)].loc[:,'Ei - Location']
                # list_country=ecoinvent_data.loc[(ecoinvent_data['Activity Name']==activity) & (ecoinvent_data['Reference Product Name']==product_name)].loc[:,'Geography']
                for country in data:
                    if country not in list_country.values:
                        print( 'PROBLEM ' + country + ' ' + activity + ' ' + str(product_name.values[0]))
                        pb=True
    
        
        if act_fill_later != []:
            i=0
            for activity in act_fill_later:
                product_name=prod_later[i]
                unit=unit_later[i]
                index=data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==activity].index
                data_LCI_PRi_model.loc[index,'Ei - Reference product']=product_name
                data_LCI_PRi_model.loc[index,'Unit']=unit
                data=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']==activity) & (data_LCI_PRi_model['Ei - Reference product']==product_name)].loc[:,'Ei - Location']
                list_country=ecoinvent_data.loc[(ecoinvent_data['Activity Name']==activity) & (ecoinvent_data['Reference Product Name']==product_name)].loc[:,'Geography']
                i+=1
                for country in data:
                    if country not in list_country.values:
                        print( 'PROBLEM ' + country + ' ' + activity + ' ' + str(product_name))
                        pb=True
        
        # if pb == False:
        #     print ('There is no problem in the country attribution') 
        
        data_LCI_PRi_model_ef=data_LCI_PRi_model.loc[data_LCI_PRi_model['Elementary flow']==data_LCI_PRi_model['Elementary flow']]
        list_ef=data_LCI_PRi_model_ef['Elementary flow'].drop_duplicates()
        unit_later_ef=[]
        ef_fill_later=[]

        
        for ef in list_ef:
            unit_ef=ei_ef_data.loc[ei_ef_data['Name']==ef].loc[:,'Unit Name'].drop_duplicates() 
            index_ef=data_LCI_PRi_model.loc[data_LCI_PRi_model['Elementary flow']==ef].index

            
            if len(unit_ef)==0:
                print( "FOR THIS ELEMENTARY FLOW: " + str(ef)+ " THERE IS NO CORRESPONDING IN THE ECOINVENT DATABASE")
                # pb=True

            if len(unit_ef)>1:
                list_dict_unit=list(dict(unit_ef).values())
        
                sel_unit=widgets.RadioButtons(
                    options=list_dict_unit,
                    #    value='pineapple', # Defaults to 'pineapple'
                    #    layout={'width': 'max-content'}, # If the items' names are long
                        description="Choose the best option for the elementary flow:" + str(ef),
                        disabled=False,
                        layout={'width': 'max-content'}
                    )
        
                display(sel_unit)
                ef_fill_later.append(ef)

                
        
            if len(unit_ef)>1:
                button = widgets.Button(description = "Click when good")
                output = widgets.Output()
                display(button, output)
                #global clicked
                clicked = False
                # def on_button_clicked(b):
                #     global clicked
                #     clicked = True
                #     with output:
                #         print("button clicked")
                
                button.on_click(on_button_clicked)

                #print(clicked)
                with ui_events() as poll:
                    while not clicked:
                        #print(clicked)
                        poll(10) # poll queued UI events including button
                        time.sleep(1) # wait for 1 second before checking again
                
                unit_later_ef.append(sel_unit.value)
                #prod_later.append(sel_prod.value)
                # print('python waited for the button click')
                
            if len(unit_ef)==1:
                data_LCI_PRi_model.loc[index_ef,'Unit']=dict_unit[str(unit_ef.values[0])]

        if unit_later_ef != []:
            i=0
            for ef in ef_fill_later:
                #product_name=prod_later[i]
                unit=unit_later_ef[i]
                index=data_LCI_PRi_model.loc[data_LCI_PRi_model['Elementary flow']==ef].index
                #data_LCI_PRi_model.loc[index,'Ei - Reference product']=product_name
                data_LCI_PRi_model.loc[index,'Unit']=unit
                # data=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']==activity) & (data_LCI_PRi_model['Ei - Reference product']==product_name)].loc[:,'Ei - Location']
                # list_country=ecoinvent_data.loc[(ecoinvent_data['Activity Name']==activity) & (ecoinvent_data['Reference Product Name']==product_name)].loc[:,'Geography']
                i+=1
                # for country in data:
                #     if country not in list_country.values:
                #         print( 'problem ' + country + ' ' + activity + ' ' + str(product_name))
                #         pb=True
    
        #unit=ecoinvent_data.loc[ecoinvent_data['Activity Name']==activity].loc[:,'Unit'].drop_duplicates()

        data_LCI_PRi_model_NoAct_NoEF=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']!=data_LCI_PRi_model['Ei - Activity name'])
                                                                &(data_LCI_PRi_model['Elementary flow']!=data_LCI_PRi_model['Elementary flow'])]  

        flow_name_noEF_noact=data_LCI_PRi_model_NoAct_NoEF.loc[:,'Flow\n(As reported by the authors)'].drop_duplicates()

        
        for flow in flow_name_noEF_noact:
            index_noef_noAct=data_LCI_PRi_model.loc[data_LCI_PRi_model['Flow\n(As reported by the authors)']==flow].index
            data_LCI_PRi_model.loc[index_noef_noAct,'Ei - Activity name']='n/a ('+ flow +')'

        if unit_later_ef != []:
            i=0
            for ef in ef_fill_later:
                #product_name=prod_later[i]
                unit=unit_later_ef[i]
                index=data_LCI_PRi_model.loc[data_LCI_PRi_model['Elementary flow']==ef].index
                #data_LCI_PRi_model.loc[index,'Ei - Reference product']=product_name
                data_LCI_PRi_model.loc[index,'Unit']=unit
                # data=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']==activity) & (data_LCI_PRi_model['Ei - Reference product']==product_name)].loc[:,'Ei - Location']
                # list_country=ecoinvent_data.loc[(ecoinvent_data['Activity Name']==activity) & (ecoinvent_data['Reference Product Name']==product_name)].loc[:,'Geography']
                i+=1
                # for country in data:
                #     if country not in list_country.values:
                #         print( 'problem ' + country + ' ' + activity + ' ' + str(product_name))
                #         pb=True    

        
        if pb == False:
            print ('There is no problem in the country attribution + in the names of activity and elementary flows') 
        
        for flow_type in dict_flow_efcat:
            data_LCI_PRi_model.loc[data_LCI_PRi_model['Classification of flows']==flow_type,'Elementary flow category']=dict_flow_efcat[flow_type]#data_LCI_PRi_model.loc[data_LCI_PRi_model['Classification of flows']==flow_type]['Elementary flow category'].replace(flow
        
        #calculate the pedigree score

        data_LCI_PRi_model["pedigree"]=('(' + data_LCI_PRi_model["Pedigree - Reliability"].astype(str)+ ',' + data_LCI_PRi_model["Pedigree - Completeness"].astype(str)+ ',' +
                                        data_LCI_PRi_model[ "Pedigree - Temporal correlation"].astype(str)+ ',' + data_LCI_PRi_model["Pedigree - Geographical correlation"].astype(str) + ',' + 
                                        data_LCI_PRi_model[ "Pedigree - Further technological correlation"].astype(str)+ ')')

     

        
        if pb ==False:
            with pd.ExcelWriter(path_step2,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:  
                data_LCI_PRi_model.to_excel(writer, sheet_name=sheet,index=False) #'PR' + str(num_PR) + '_LCI_Modelling'
    
        #num_PR+=1



NameError: name 'path_mapping' is not defined

In [ ]:
#num_PR=1

columns_name_BW=["name", "reference product",	"location",	"amount",	"unit",	"database",	"type",	"categories",	"uncertainty type",
                  "pedigree" ,"loc",	"scale", "minimum", "maximum"]


def createBW(sheet_names_list_model,num_PR=1):
    for sheet in sheet_names_list_model:
    
    
        data_activity_blue=pd.read_excel(path_step2,sheet_name=sheet) #'PR'+ str(num_PR) +'_LCI_Modelling'
        
        print( 'The production route is ' + str(sheet))
    
        
        unit_ope_list=list(data_activity_blue.loc[:,'Associated unit operation\n(as reported by the authors)'].drop_duplicates().apply(str))
        lines_name_BW=["Activity", "reference product", "location", "amount", "unit", "comment", "Exchanges"]
        last_ope=False
        
        #columns_name_BW=["name", "reference product",	"location",	"amount",	"unit",	"database",	"type",	"categories",	"uncertainty type",
        #                 "loc",	"scale", "minimum", "maximum"]
        
        start_row=0
        
        while last_ope!=True:
        
            activity=widgets.Text(value='', disabled=False,description='Activity name') # Value saved here
            ref_prod=widgets.Text(value='', disabled=False,description='Reference product')
            location=widgets.Text(value='', disabled=False,description='Location')
            amount=widgets.Text(value='', disabled=False,description='Amount')
            unit=widgets.Dropdown(options=list(dict_unit.values()),description='Unit',disabled=False)
            #widgets.Text(value='', disabled=False ,description='Unit')
            comment=widgets.Text(value='n/a', disabled=False ,description='Comment')
            exchanges=widgets.Text(value='', disabled=False ,description='Exchanges')
        
            display(activity)
            display(ref_prod)
            display(location)
            display(amount)
            display(unit)
            display(comment)
            #display(exchanges)
        
        
        
            checkboxes = [widgets.Checkbox(value=False, description=label) for label in unit_ope_list]
            output = widgets.VBox(children=checkboxes)
            display(output)
        
        
            var1=widgets.Checkbox(
                value=False,
                description='If it is the last operation',
                disabled=False,
                indent=True)
        
            display(var1)
        
        
            button = widgets.Button(description = "Click when good")
            output = widgets.Output()
            display(button, output)
            global clicked    
            clicked = False
            # def on_button_clicked(b):
            #     global clicked
            #     clicked = True
            #     with output:
            #         print("button clicked")
                
            button.on_click(on_button_clicked)
                
            with ui_events() as poll:
                while not clicked:
                    poll(10) # poll queued UI events including button
                    time.sleep(1) # wait for 1 second before checking again
                
        
                # print('python waited for the button click')
            
           
            
            n=len(unit_ope_list)
    
            
            act=activity.value
            ref=ref_prod.value
            loc=location.value
            amount_val=amount.value
            unit_val=unit.value
            com=comment.value
            ex=exchanges.value
            
            last_ope=var1.value
            
            list_ope= []
            for i in range(0, len(checkboxes)):
                if checkboxes[i].value == True:
                    list_ope = list_ope + [checkboxes[i].description]
    
            if com=='':
                com='n/a'
            
            df = pd.DataFrame(lines_name_BW)
            df['col2']=[act,ref,loc,amount_val,unit_val,com, ex]
            df=df.rename(columns=df.iloc[0])
            df=df.drop(df.index[0])
        
            with pd.ExcelWriter(path_step3,engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer:  
                df.to_excel(writer, sheet_name=sheet,index=False,header=True,startrow=start_row)#'PR' + str(num_PR) + '_LCI_BW'
        
        
        
                #writer.save()
            start_row+=7
        
            # list_ope=[]
            # for el in boxes: #to have the list of operation unit for this activity
            #     if el[1].get()==1:
            #         list_ope.append(el[0])
        
            #add first inventory
            
            #data_activity=pd.read_excel(path_LCI_BW,header=None,index_col=0)
            
            line1=[act,ref,loc,amount_val,unit_val, '', 'production', '', '' ,'','','','','']
            data_LCI_activity_BW=pd.DataFrame(data=[line1],columns=columns_name_BW)
            
            
            data_selected=data_activity_blue.loc[data_activity_blue['Associated unit operation\n(as reported by the authors)'].isin(list_ope)].copy() # pour selectionner les lines qui ont les étapes demandées 
            #data_activity_blue
            col_need_BW_activity=['Ei - Activity name','Ei - Reference product', 'Ei - Location', "modified value allocated flow",'Unit', 'pedigree']
            data_activity_ei_selec=data_selected.loc[:,col_need_BW_activity].dropna(subset=['Ei - Activity name']).reset_index(drop=True).copy()
            
            data_activity_ei_selec.columns=["name", "reference product",	"location",	"amount",	"unit",'pedigree']
            data_activity_BW=pd.DataFrame(data=data_activity_ei_selec,columns=columns_name_BW)
            data_activity_BW['database']='ecoinvent 3.10 cutoff'
            data_activity_BW['type']='technosphere'


            col_need_BW_flow_with_subcat=['Elementary flow','Elementary flow category', "Elementary flow sub-category", 'Ei - Location', "modified value allocated flow",'Unit',"pedigree"]
            #!! need to change elementary flow category with :: and sub category
            data_flow_selec_with_subcat=data_selected.loc[:,col_need_BW_flow_with_subcat].dropna(subset=['Elementary flow']).reset_index(drop=True).copy()
            for ind in data_flow_selec_with_subcat.index:
                if data_flow_selec_with_subcat.loc[ind,"Elementary flow sub-category"]==data_flow_selec_with_subcat.loc[ind,"Elementary flow sub-category"]:
                    data_flow_selec_with_subcat.loc[ind,"Elementary flow category"]=(data_flow_selec_with_subcat.loc[ind,"Elementary flow category"]+
                                                                                     '::'+data_flow_selec_with_subcat.loc[ind,"Elementary flow sub-category"])

            col_need_BW_flow=['Elementary flow','Elementary flow category', 'Ei - Location', "modified value allocated flow",'Unit',"pedigree"]
            #!! need to change elementary flow category with :: and sub category
            data_flow_selec=data_flow_selec_with_subcat.loc[:,col_need_BW_flow].dropna(subset=['Elementary flow']).reset_index(drop=True).copy()
            data_flow_selec.columns=["name", "categories",	"location",	"amount",	"unit","pedigree"]
            
            
            # col_need_BW_flow=['Elementary flow','Elementary flow category', 'Ei - Location', "modified value allocated flow",'Unit',"pedigree"]
            # #!! need to change elementary flow category with :: and sub category
            # data_flow_selec=data_selected.loc[:,col_need_BW_flow].dropna(subset=['Elementary flow']).reset_index(drop=True).copy()
            # data_flow_selec.columns=["name", "categories",	"location",	"amount",	"unit","pedigree"]
            
            data_flow_BW=pd.DataFrame(data=data_flow_selec,columns=columns_name_BW)
            data_flow_BW['database']='biosphere3'
            data_flow_BW['type']='biosphere'
            
            all_pd=[data_LCI_activity_BW,data_activity_BW,data_flow_BW]
            data_BW=pd.concat(all_pd,ignore_index=True)
        
            with pd.ExcelWriter(path_step3,engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer:  #, engine='xlsxwriter'
               data_BW.to_excel(writer, sheet_name=sheet,index=False,startrow=start_row)#'PR' + str(num_PR) + '_LCI_BW'
        
        
        
        
            start_row=start_row+len(data_BW)+2
        
            if last_ope==1:    
                last_ope= True
    
    
        #num_PR+=1


In [6]:

columns_name_BW=["name", "reference product",	"location",	"amount",	"unit",	"database",	"type",	"categories",	"uncertainty type",
                  "pedigree" ,"loc",	"scale", "minimum", "maximum"]
len(columns_name_BW)

line1=['act','ref','loc','amount_val','unit_val', '', 'production', '', '' ,'','','','','']
len(line1)
data_LCI_activity_BW=pd.DataFrame(data=[line1],columns=columns_name_BW)